In [2]:
# import mods
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import find_peaks

In [124]:
# data processing
def read_data(filename):
    data = pd.read_csv(filename, header = 18, skip_blank_lines = False, usecols = [0, 1, 2])
    specs = pd.read_csv(filename, header = 8, skip_blank_lines = False, nrows = 2)
    width = pd.to_numeric(specs.loc[1][1])
    height = pd.to_numeric(specs.loc[1][2])
    data['Stress'] = pd.to_numeric(data.loc[1:,'Force'])*1000/(width*height)
    data['Stroke Strain (%)'] = pd.to_numeric(data.loc[1:,'Stroke'])/60*100
    data['dy'] = data['Stress'].diff()
    data['dx'] = data['Stroke Strain (%)'].diff()
    data['gradient'] = data['dy']/data['dx']
    data = data.drop(0)
    return data

In [125]:
# check data
print(read_data("CF1.csv"))

      Time       Force Stroke      Stress  Stroke Strain (%)        dy  \
1        0  -0.0003125      0   -0.017299           0.000000       NaN   
2     0.05   -0.000625      0   -0.034599           0.000000 -0.017299   
3      0.1   0.0009375  0.004    0.051898           0.006667  0.086497   
4     0.15   0.0084375  0.012    0.467084           0.020000  0.415186   
5      0.2   0.0234375   0.02    1.297456           0.033333  0.830372   
..     ...         ...    ...         ...                ...       ...   
508  25.35    6.242188  4.212  345.555740           7.020000  0.830372   
509   25.4    6.256875   4.22  346.368785           7.033333  0.813045   
510  25.45    6.269063  4.228  347.043489           7.046667  0.674705   
511  25.45    6.269063  4.228  347.043489           7.046667  0.000000   
512  25.45    6.269063  4.228  347.043489           7.046667  0.000000   

           dx   gradient  
1         NaN        NaN  
2    0.000000       -inf  
3    0.006667  12.974557  
4  

In [75]:
# draw figures
# colours are dark to light

color_blues = ['#004d99','#0066cc','#0080ff','#3399ff','#66b3ff', '#99ccff']
color_reds = ['#560319', '#940000', '#b30000', '#cc0000', '#ff4747', '#f08080']

def drawfig(filename, fig, to_plot, color):
    df = read_data(filename)

    fig.add_trace(
        go.Scatter(
            x = df["Stroke Strain (%)"],
            y = to_plot,
            mode = "lines",
            line_color = color[int(filename[2:-4])],
            name = filename[:-4],
            showlegend = True
        )
    )
    return

In [119]:
# format figures
files = ["CF1.csv","CF2.csv","CF3.csv","CF4.csv","CF5.csv"]
files2 = ["GF1.csv","GF2.csv","GF3.csv","GF4.csv","GF5.csv"]

def fig_draw(status, array, pic_name, title_name, to_plot, color, save):
    # use this for combined graphs
    if status == 'multiple':
        fig = go.Figure()

    for i in array:
        # use this fig for indiv graphs
        if status == 'indiv':
            fig = go.Figure()
        
        for j in to_plot:
            drawfig(i, fig, read_data(i)[j], color)

        if status == 'indiv':
            fig.update_layout(
                title = i[:-4]
            )
        
        if status == 'multiple':
            fig.update_layout(
                title = title_name
            )

        fig.update_layout(
        xaxis = dict(
            title = "Stroke Strain (%)",
            showline = True,
            linecolor = "black",
            linewidth = 2,
            mirror = "ticks",
            ticks = "inside",
            showticklabels = True),

        yaxis = dict(
            title = "Stress (N/mm2)",
            showline = True,
            linecolor = "black",
            linewidth = 2,
            mirror = "ticks",
            ticks = "inside",
            showticklabels = True
        ),

        plot_bgcolor = "white",

        font = dict(
            family = "Times New Roman"
            )
    )
        if status == 'indiv':
            # for indiv graph
            fig.show()

            if save:
                fig.write_image(i[:-4] + ".png")

    if status == 'multiple':
        # toggle for combined graphs
        fig.show()
        
        if save:
            fig.write_image(pic_name + ".png")
    return

In [126]:
fig_draw('indiv', files2, 'test', 'testing', ['gradient', 'Stress'], color_reds, False)

fig_draw('multiple', files2, 'test', 'testing', ['gradient', 'Stress'], color_reds, False)

In [27]:
# find Ultimate Tensile Strengths & Stroke Strain
all_files = files + files2
def calculate_max(parameter):
    for i in all_files:
        df = read_data(i)
        print(df[parameter].max(), i)
    return

calculate_max("Stress")
calculate_max("Stroke Strain (%)")

347.04348933249184 CF1.csv
306.2622254823509 CF2.csv
280.8515014397367 CF3.csv
369.7812580719715 CF4.csv
365.15605788466877 CF5.csv
146.73136073616516 GF1.csv
162.83560390227058 GF2.csv
176.16334283000947 GF3.csv
160.90863060409396 GF4.csv
166.55786421453976 GF5.csv
7.046666666666666 CF1.csv
6.114999999999999 CF2.csv
5.088333333333333 CF3.csv
7.598333333333334 CF4.csv
7.098333333333333 CF5.csv
5.423333333333333 GF1.csv
4.991666666666667 GF2.csv
5.673333333333333 GF3.csv
4.801666666666668 GF4.csv
5.410000000000001 GF5.csv


In [102]:
# find the max gradient, the point it's at, and the yield stress
for i in all_files:
    all_data = read_data(i)
    df = read_data(i)['instantaneous_gradient']
    max_gradient = df.max()
    max_gradient_index = df.idxmax()
    yield_stress = all_data.at[max_gradient_index, 'Stress']

    print (max_gradient, i, max_gradient_index, yield_stress)

1.4012522004849366 CF1.csv 30 28.146139878876447
1.4934538099048638 CF2.csv 38 39.10765092913667
1.4397367338543816 CF3.csv 31 29.651720828191415
1.121579183490356 CF4.csv 24 16.63675788844026
1.478512288079905 CF5.csv 21 17.43184240884333
0.913533643458146 GF1.csv 7 2.452639238414805
1.0144245877579188 GF2.csv 17 13.414058534058533
0.9364777883296398 GF3.csv 8 3.5744433892582035
1.0504885822395993 GF4.csv 8 3.821152217896543
1.126405754381468 GF5.csv 8 4.780966646374676
